In [1]:
from datasets import load_dataset
dataset_full = load_dataset("mbpp")

/home/hongyihuang/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating prompt split: 100%|██████████| 10/10 [00:00<00:00, 9360.20 examples/s]


In [7]:
dataset_full

DatasetDict({
    train: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 374
    })
    test: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 500
    })
    validation: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 90
    })
    prompt: Dataset({
        features: ['task_id', 'text', 'code', 'test_list', 'test_setup_code', 'challenge_test_list'],
        num_rows: 10
    })
})

In [8]:
dataset_full['train']['test_list'][0]

['assert max_chain_length([Pair(5, 24), Pair(15, 25),Pair(27, 40), Pair(50, 60)], 4) == 3',
 'assert max_chain_length([Pair(1, 2), Pair(3, 4),Pair(5, 6), Pair(7, 8)], 4) == 4',
 'assert max_chain_length([Pair(19, 10), Pair(11, 12),Pair(13, 14), Pair(15, 16), Pair(31, 54)], 5) == 5']

In [9]:
dataset_full['train']['text'][0]

'Write a function to find the longest chain which can be formed from the given set of pairs.'

In [10]:
dataset_full['train']['code'][0]

'class Pair(object): \r\n\tdef __init__(self, a, b): \r\n\t\tself.a = a \r\n\t\tself.b = b \r\ndef max_chain_length(arr, n): \r\n\tmax = 0\r\n\tmcl = [1 for i in range(n)] \r\n\tfor i in range(1, n): \r\n\t\tfor j in range(0, i): \r\n\t\t\tif (arr[i].a > arr[j].b and\r\n\t\t\t\tmcl[i] < mcl[j] + 1): \r\n\t\t\t\tmcl[i] = mcl[j] + 1\r\n\tfor i in range(n): \r\n\t\tif (max < mcl[i]): \r\n\t\t\tmax = mcl[i] \r\n\treturn max'

In [14]:
[len(s) for s in dataset_full['train']['test_setup_code']]

TypeError: len() takes exactly one argument (0 given)

In [11]:
code = dataset_full['train']['code']
test = dataset_full['train']['test_list']
setup = dataset_full['train']['test_setup_code']
rows = dataset_full['train'].num_rows

#print(code)
#print(test)
#print(rows)

success_count = 0
fail_count = 0

for i in range(rows):
    success = True
    for j in range(len(test[i])):
        try:
            appended_code = code[i] + "\r\n" + setup[i] + "\r\n" + test[i][j]
            # print(appended_code)
            
            exec(appended_code, globals(), locals())
            # exec(dataset_full['train']['test_list'][0], context, local_var)
            success_count += 1
        except Exception as e:
            print("Test failed: ", i, j, test[i][j], e)
            # print("Code: ", code[i])
            fail_count += 1
            success = False
    if not success:
        print("Code: ", code[i])

print("Success: ", success_count)
print("Fail: ", fail_count)

Success:  1122
Fail:  0
